Create HeatMapWithTime for all start stations from 2011 to 2017

In [4]:
import folium
from folium.features import GeoJsonTooltip
import fiona
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geopandas.tools
from geopandas.tools import geocode
from shapely.geometry import Point, Polygon
from folium.plugins import HeatMap
import numpy as np

from folium.plugins import HeatMapWithTime
from collections import OrderedDict, defaultdict


<h3> Create HeatMapWithTime for start stations all years </h3>


In [5]:
stations_2011 = pd.read_csv('cleaned_data/stations_2011.csv')
stations_2012 = pd.read_csv('cleaned_data/stations_2012.csv')
stations_2013 = pd.read_csv('cleaned_data/stations_2013.csv')
stations_2014 = pd.read_csv('cleaned_data/stations_2014.csv')
stations_2015 = pd.read_csv('cleaned_data/stations_2015.csv')
stations_2016 = pd.read_csv('cleaned_data/stations_2016.csv')
stations_2017 = pd.read_csv('cleaned_data/stations_2017.csv')
df_2011 = pd.read_csv('cleaned_data/df_2011.csv')
df_2012 = pd.read_csv('cleaned_data/df_2012.csv')
df_2013 = pd.read_csv('cleaned_data/df_2013.csv')
df_2014 = pd.read_csv('cleaned_data/df_2014.csv')
df_2015 = pd.read_csv('cleaned_data/df_2015.csv')
df_2016 = pd.read_csv('cleaned_data/df_2016.csv')
df_2017 = pd.read_csv('cleaned_data/df_2017.csv')

In [6]:
stations_all_years = [stations_2011, stations_2012, stations_2013, stations_2014, stations_2015, stations_2016, stations_2017]
df_all_years = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017]

In [7]:
# Convert "start_date" column in df_20xx into datetime format
for df_year in df_all_years:
    df_year['start_date'] = pd.to_datetime(df_year['start_date'])

In [8]:
df_start_all_years = []

for i in range(len(df_all_years)):
    df_temp = df_all_years[i][['start_date', 'start_station', 'registered', 'casual']]
    df_temp = df_temp.groupby(['start_date', 'start_station']).sum().reset_index()
    df_temp['year'] = df_temp['start_date'].dt.year
    df_temp = df_temp.groupby(['year', 'start_station']).sum().reset_index()
    df_temp = df_temp.merge(stations_all_years[i], left_on = 'start_station', right_on = 'station').drop(['station', 'region'], axis = 1)
    df_start_all_years.append(df_temp)

df_start_2011 = df_start_all_years[0]
df_start_2012 = df_start_all_years[1]
df_start_2013 = df_start_all_years[2]
df_start_2014 = df_start_all_years[3]
df_start_2015 = df_start_all_years[4]
df_start_2016 = df_start_all_years[5]
df_start_2017 = df_start_all_years[6]


In [9]:
df_start_combined = pd.concat([df_start_2011, df_start_2012, df_start_2013, df_start_2014, df_start_2015, df_start_2016, df_start_2017])

In [10]:
from sklearn.preprocessing import minmax_scale

In [11]:
df_start_combined['registered'] = minmax_scale(df_start_combined['registered'])
df_start_combined['casual'] = minmax_scale(df_start_combined['casual'])

In [12]:
df_start_combined [df_start_combined['year']==2011].head()

,year,start_station,registered,casual,lat,lon
0,2011,10th & Monroe St NE,0.031697,0.005821,38.932457,-76.993534
1,2011,10th & U St NW,0.236608,0.032184,38.917200,-77.025900
2,2011,10th St & Constitution Ave NW,0.123232,0.153641,38.893028,-77.026013
3,2011,11th & F St NW,0.000349,0.000000,38.897857,-77.026975
4,2011,11th & H St NE,0.003286,0.000123,38.900164,-77.005441


In [13]:
df_start_combined [df_start_combined['year']==2017].head()

,year,start_station,registered,casual,lat,lon
0,2017,10th & E St NW,0.132644,0.124042,38.895914,-77.026064
1,2017,10th & Florida Ave NW,0.129454,0.014668,38.920387,-77.025672
2,2017,10th & G St NW,0.126280,0.065230,38.898243,-77.026235
3,2017,10th & K St NW,0.230687,0.085244,38.902400,-77.026220
4,2017,10th & Monroe St NE,0.047585,0.010605,38.932457,-76.993534


In [14]:
stations_start_reg = defaultdict(list)
for row in df_start_combined.itertuples():
    stations_start_reg[row.year].append([row.lat, row.lon, row.registered])
stations_start_reg = OrderedDict(sorted(stations_start_reg.items()))

In [15]:
m=folium.Map(location=[38.900497,-77.007507],zoom_start=12,min_zoom=11,zoom_control=False,Control_scale=True)
hm = HeatMapWithTime(data=list(stations_start_reg.values()),
                     index=list(stations_start_reg.keys()), 
                     radius=20,
                     auto_play=False,
                     max_opacity=0.5)
hm.add_to(m)
m